# 뉴스 크롤링

In [ ]:
pip install selenium

In [ ]:
pip install webdriver-manager

In [ ]:
!apt-get update
!apt install chromium-chromedriver

In [16]:
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import re
import datetime
from pytz import timezone
KST = timezone('Asia/Seoul')

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
browser = webdriver.Chrome(options=options)

## 기사 링크 크롤링

In [34]:
# 기사 링크 크롤링
# 카테고리당 100개 = 5페이지

url_list = []   # url 저장 리스트

category_list = []
categoty_name = ["정치", "경제", "사회", "생활/문화", "세계", "IT/과학"]

def getLink():    # 정치, 경제, 사회, 생활/문화, 세계, IT/과학
  a_list = []

  for category in range(6):     # 6
      for page in range(1, 6):  # 1, 6
          url = f'https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1={100 + category}#&date=%2000:00:00&page={page}'
          browser.get(url)

          time.sleep(1)

          soup = BeautifulSoup(browser.page_source, "html.parser")
          a_list.extend(soup.select(".type06_headline dt+dt a"))
          a_list.extend(soup.select(".type06 dt+dt a"))

          category_list.extend([categoty_name[category] for _ in range(20)])
          print(f"{categoty_name[category]} {page} 페이지")

  for a in a_list:
    url_list.append(a["href"])


def getEntertainmentLink():   # 연예
  # today = str(datetime.datetime.now(KST))[:11]  # 서울 기준 시간
  a_list = []
  today = datetime.date.today()

  for page in range(1, 2):  # 1, 5
      url = f'https://entertain.naver.com/now#sid=106&date={today}&page={page}'
      browser.get(url)

      time.sleep(1)

      soup = BeautifulSoup(browser.page_source, "html.parser")

      a_list.extend(soup.select(".news_lst li>a"))

      category_list.extend(["연예" for _ in range(25)])
      print(f"연예 {page} 페이지")

  for a in a_list:
    url_list.append("https://entertain.naver.com" + a["href"])


def getSportsLink():    # 스포츠  (페이지마다 개수가 달라서 6페이지를 이동)
  # today = str(datetime.datetime.now(KST))[:11].replace('-', '')  # 서울 기준 시간
  a_list = []
  today = str(datetime.date.today()).replace('-', '')
  for page in range(1, 7):  # 1, 7
      url = f'https://sports.news.naver.com/general/news/index?isphoto=N&type=latest&date={today}&page={page}'
      browser.get(url)

      time.sleep(1)

      soup = BeautifulSoup(browser.page_source, "html.parser")
      a_list.extend(soup.select(".news_list li>a"))

      print(f"스포츠 {page} 페이지")

  for i in range(len(a_list)):
    if i == 100:  # 100개 링크 추가했으면 멈추기
      break
    url_list.append("https://sports.news.naver.com/news" + re.search('\?.+', a_list[i]["href"]).group())
    category_list.extend(["스포츠"])

# getLink()
getEntertainmentLink()
# getSportsLink()


연예 1 페이지


In [35]:
print(len(url_list))
print(len(category_list))
print(url_list)

25
25
['https://entertain.naver.com/now/read?oid=109&aid=0004957664', 'https://entertain.naver.com/now/read?oid=241&aid=0003309895', 'https://entertain.naver.com/now/read?oid=108&aid=0003191894', 'https://entertain.naver.com/now/read?oid=640&aid=0000045305', 'https://entertain.naver.com/now/read?oid=109&aid=0004957663', 'https://entertain.naver.com/now/read?oid=241&aid=0003309894', 'https://entertain.naver.com/now/read?oid=241&aid=0003309893', 'https://entertain.naver.com/now/read?oid=468&aid=0000996931', 'https://entertain.naver.com/now/read?oid=144&aid=0000922403', 'https://entertain.naver.com/now/read?oid=108&aid=0003191892', 'https://entertain.naver.com/now/read?oid=109&aid=0004957662', 'https://entertain.naver.com/now/read?oid=640&aid=0000045303', 'https://entertain.naver.com/now/read?oid=241&aid=0003309892', 'https://entertain.naver.com/now/read?oid=421&aid=0007148458', 'https://entertain.naver.com/now/read?oid=311&aid=0001656566', 'https://entertain.naver.com/now/read?oid=109&ai

## 제목, 본문, 날짜 크롤링

In [96]:
# 수집한 태그 속에서 내용만 뽑아서 담는다.

def clean(article):   # 전처리
  article = re.sub('\n','',article)
  article = re.sub('\t','',article)
  article = re.sub('\u200b','',article)
  article = re.sub('\xa0','',article)
  article = re.sub('([a-zA-Z])','',article)
  article = re.sub('[ㄱ-ㅎㅏ-ㅣ]+','',article)
  article = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','',article)
  article = re.sub('[ㄱ-ㅎㅏ-ㅣ]+','',article)
  article = re.sub('.{2,3} 기자','',article)


  return article


In [97]:
# 수집한 링크의 제목, 본문, 날짜 크롤링

title_list = []     # 제목
content_list = []   # 본문
date_list = []      # 날짜

def getContent():   # 정치, 경제, 사회, 생활/문화, 세계, IT/과학
  title = []
  content = []
  date = []

  for url in url_list:
        browser.get(url)

        time.sleep(1)

        soup = BeautifulSoup(browser.page_source, "html.parser")

        title.extend(soup.select("#title_area span"))                 # 제목

        c = soup.find_all(attrs={"id" : "dic_area"})                  # 본문

        while c[0].find(attrs={"class" : "end_photo_org"}):           # 이미지 있는 만큼
          c[0].find(attrs={"class" : "end_photo_org"}).decompose()    # 본문 이미지에 있는 글자 없애기

        while c[0].find(attrs={"class" : "vod_player_wrap"}):         # 영상 있는 만큼
          c[0].find(attrs={"class" : "vod_player_wrap"}).decompose()  # 본문 영상에 있는 글자 없애기

        content.extend(c)

        date.extend(soup.select("._ARTICLE_DATE_TIME"))               # 날짜

  for t in title:
    title_list.append(t.text)

  for c in content:
    content_list.append(clean(c.text))

  for d in date:
    date_list.append(d["data-date-time"])


def getEntertainmentContent():    # 연예
    title = []
    content = []
    date = []

    for url in url_list:
        browser.get(url)

        time.sleep(1)

        soup = BeautifulSoup(browser.page_source, "html.parser")

        title.extend(soup.select(".end_tit"))                 # 제목

        c = soup.find_all(attrs={"id" : "articeBody"})                # 본문

        while c[0].find(attrs={"class" : "end_photo_org"}):           # 이미지 있는 만큼
          c[0].find(attrs={"class" : "end_photo_org"}).decompose()    # 본문 이미지에 있는 글자 없애기
          if c[0].find(attrs={"class" : "caption"}):                  # 이미지 설명 없애기
            c[0].find(attrs={"class" : "caption"}).decompose()

        while c[0].find(attrs={"class" : "video_area"}):         # 영상 있는 만큼
          c[0].find(attrs={"class" : "video_area"}).decompose()  # 본문 영상에 있는 글자 없애기

        while c[0].find(attrs={"name" : "iframe"}):
           c[0].find(attrs={"name" : "iframe"}).decompose()

        content.extend(c)

        date.extend(soup.select_one(".author em"))               # 날짜

    for t in title:
      title_list.append(clean(t.text))

    for c in content:
      content_list.append(clean(c.text))

    for d in date:
      date_list.append(d.text)


def getSportsContent():   # 스포츠
  pass

# getContent()
getEntertainmentContent()


In [67]:
print(len(title_list))
print(len(content_list))
print(len(date_list))

25
25
25
['2023.11.01. 오후 7:40', '2023.11.01. 오후 7:38', '2023.11.01. 오후 7:38', '2023.11.01. 오후 7:38', '2023.11.01. 오후 7:38', '2023.11.01. 오후 7:37', '2023.11.01. 오후 7:37', '2023.11.01. 오후 7:37', '2023.11.01. 오후 7:37', '2023.11.01. 오후 7:36', '2023.11.01. 오후 7:36', '2023.11.01. 오후 7:35', '2023.11.01. 오후 7:34', '2023.11.01. 오후 7:34', '2023.11.01. 오후 7:34', '2023.11.01. 오후 7:33', '2023.11.01. 오후 7:32', '2023.11.01. 오후 7:32', '2023.11.01. 오후 7:31', '2023.11.01. 오후 7:31', '2023.11.01. 오후 7:30', '2023.11.01. 오후 7:30', '2023.11.01. 오후 7:29', '2023.11.01. 오후 7:29', '2023.11.01. 오후 7:27']


## 데이터 프레임 생성

In [ ]:
# 데이터 프레임으로 변환

article_df = pd.DataFrame({"카테고리" : category_list,
                           "날짜" : date_list,
                           "제목" : title_list,
                           "본문" : content_list,
                           "링크" : url_list})

article_df

# 전처리

**KoNLPy**
- 한국어 정보처리를 위한 파이썬 패키지
- https://konlpy.org/ko/latest/api/konlpy.tag/#okt-class

In [ ]:
pip install konlpy

In [ ]:
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# CountVectorizer : 텍스트 빈도만을 카운트해서 벡터화
# TfidfVectorizer : 자주 나오는 단어에 높은 가중치, 모든 문서에서 자주 나오는 단어에 패널티

In [ ]:
okt = Okt()
nouns_list = []   # 명사 리스트

for content in article_df["본문"]:
  nouns_list.append(okt.nouns(content))     # 명사 추출, 리스트 반환

article_df["명사"] = nouns_list   # 데이터 프레임에 추가

In [ ]:
text = [" ".join(noun) for noun in article_df["명사"]]  # 명사 열을 하나의 리스트에 담는다.

tfidf_vectorizer = TfidfVectorizer(min_df = 5, ngram_range=(1, 5))

# 군집화

In [ ]:
from sklearn.cluster import DBSCAN
# DBSCAN : 밀도 기반 클러스터링
# https://hoonzi-text.tistory.com/19